In [2]:
from datetime import datetime, date, timedelta
import psycopg2 as pc
from psycopg2.extras import DictCursor, execute_values
import pandas as pd

# Set connection with the server

In [3]:
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "cosmetic_store"
DB_USER = "postgres"
DB_PASSWORD = ""

In [4]:
connection = pc.connect(
    host=DB_HOST,
    port=DB_PORT,
    database=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD
)

# Check that connection is working

In [5]:
query = """
SELECT current_date;
"""

In [6]:
# create connection cursor
cursor = connection.cursor()
# execute query
cursor.execute(query)
# fetch result into dataframe
df = pd.DataFrame(cursor.fetchone())
# get column names from cursor description
df.columns = [desc[0] for desc in cursor.description]
cursor.close()

In [7]:
df

,current_date
0,2023-05-29


## As the table that do not have any foreign key is base of all the products, start with it

In [8]:
product_filename = 'product_data.csv'
df_product = pd.read_csv(product_filename)

In [9]:
df_product.head()

,id,product_name,category_1,category_2,category_3,brand
0,1989,Coty brand collaborative networks 78ml,Fragrance,Conditioners,Night Creams,Coty
1,1843,Coty enable B2C e-markets 496ml,Makeup,Body Lotion,Face Cleansers,Estée Lauder
2,2763,Estée Lauder visualize holistic interfaces 415ml,Bath & Body,Lipstick,Night Creams,Unilever
3,2925,Procter & Gamble redefine seamless application...,Makeup,Mascara,Matte Lipstick,Unilever
4,1251,Coty repurpose transparent technologies 387ml,Skin Care,Masks,Matte Lipstick,Estée Lauder


In [10]:
quary_instert_into_product = '''
    INSERT INTO public.product (id, title, category_1, category_2, category_3, brand)
    VALUES (%(id)s, %(title)s, %(category_1)s, %(category_2)s, %(category_3)s, %(brand)s)
    
    ON CONFLICT (id)
    DO UPDATE SET
        title = EXCLUDED.title,
        category_1 = EXCLUDED.category_1,
        category_2 = EXCLUDED.category_2,
        category_3 = EXCLUDED.category_3,
        brand = EXCLUDED.brand
    RETURNING id;
'''

In [11]:
with connection:
    with connection.cursor() as cursor:
        count = 0
        for purchase_id, purchase_title, purchase_category_1, purchase_category_2, purchase_category_3, purchase_brand in df_product.values:
            
            data_keys = {
                'id': purchase_id,
                'title': purchase_title,
                'category_1': purchase_category_1,
                'category_2': purchase_category_2,
                'category_3': purchase_category_3,
                'brand': purchase_brand
            }
            try:
                cursor.execute(quary_instert_into_product, data_keys)
        
                #print(f"New ID: {cursor.fetchone()[0]}")
            except Exception as e:
                print(count, purchase_id, purchase_title, f"Error: {e}")
                break;
            count +=1

## Another table that do not have foreign keys is chops, but in the given data there are not information about shops, so i took it from purchases records.

In [12]:
shops_filename = 'synthetic_data_shops.csv'
df_shops = pd.read_csv(shops_filename)

In [13]:
df_shops.head()

,ID,Address
0,A60,"Vienna, Elm Street 64935"
1,B74,"Amsterdam, Fleet Street 781"
2,A08,"Warsaw, Fleet Street 4505"
3,A22,"Rome, Oxford Street 1150"
4,B76,"Lisbon, Main Street 57581"


In [14]:
shops2 = df_shops['ID'].unique()
shops2 = set(shops2)
shops2

{'A02',
 'A03',
 'A04',
 'A05',
 'A06',
 'A07',
 'A08',
 'A09',
 'A10',
 'A11',
 'A12',
 'A13',
 'A14',
 'A15',
 'A16',
 'A17',
 'A18',
 'A19',
 'A21',
 'A22',
 'A25',
 'A26',
 'A27',
 'A28',
 'A29',
 'A30',
 'A31',
 'A32',
 'A33',
 'A34',
 'A36',
 'A39',
 'A40',
 'A41',
 'A42',
 'A43',
 'A44',
 'A46',
 'A47',
 'A48',
 'A49',
 'A50',
 'A51',
 'A52',
 'A54',
 'A55',
 'A56',
 'A57',
 'A58',
 'A59',
 'A60',
 'A61',
 'A62',
 'A63',
 'A65',
 'A66',
 'A67',
 'A68',
 'A69',
 'A70',
 'A72',
 'A73',
 'A75',
 'A76',
 'A79',
 'A80',
 'A81',
 'A82',
 'A83',
 'A84',
 'A85',
 'A86',
 'A87',
 'A88',
 'A89',
 'A90',
 'A92',
 'A93',
 'A94',
 'A95',
 'A96',
 'A98',
 'A99',
 'B01',
 'B02',
 'B03',
 'B04',
 'B05',
 'B07',
 'B08',
 'B09',
 'B10',
 'B11',
 'B12',
 'B14',
 'B15',
 'B16',
 'B17',
 'B18',
 'B21',
 'B22',
 'B24',
 'B26',
 'B27',
 'B31',
 'B32',
 'B42',
 'B44',
 'B45',
 'B46',
 'B47',
 'B48',
 'B49',
 'B50',
 'B51',
 'B53',
 'B54',
 'B55',
 'B56',
 'B57',
 'B58',
 'B59',
 'B60',
 'B61',
 'B64',


In [15]:
query_insert_into_shop = '''
    INSERT INTO public.shop (id, address)
    VALUES (%(id)s, %(address)s)
    RETURNING id;
'''

In [16]:
with connection:
    with connection.cursor() as cursor:
        for shop_id, shop_address in df_shops.values:
            data_keys = {
                'id': shop_id,
                'address': shop_address
            }
            try:
                cursor.execute(query_insert_into_shop, data_keys)
                print(f"New ID: {cursor.fetchone()[0]}")
            except Exception as e:
                print(f"Error: {e}")

New ID: A60
New ID: B74
New ID: A08
New ID: A22
New ID: B76
New ID: A29
New ID: A95
New ID: A03
New ID: A80
New ID: A05
New ID: A90
New ID: B55
New ID: A89
New ID: A67
New ID: B51
New ID: B01
New ID: A21
New ID: A07
New ID: B22
New ID: A49
New ID: A57
New ID: A65
New ID: A59
New ID: B32
New ID: A14
New ID: A52
New ID: B08
New ID: A26
New ID: A68
New ID: A85
New ID: B50
New ID: A55
New ID: A09
New ID: A42
New ID: A41
New ID: A99
New ID: B64
New ID: A11
New ID: B09
New ID: A83
New ID: A12
New ID: A81
New ID: B60
New ID: B57
New ID: B83
New ID: A86
New ID: A82
New ID: B82
New ID: B02
New ID: A13
New ID: A93
New ID: A06
New ID: B16
New ID: A25
New ID: B45
New ID: B24
New ID: A44
New ID: B14
New ID: B54
New ID: B31
New ID: A02
New ID: B46
New ID: A47
New ID: B07
New ID: A43
New ID: A79
New ID: A48
New ID: B67
New ID: B12
New ID: B26
New ID: A40
New ID: B48
New ID: B10
New ID: A88
New ID: B04
New ID: A51
New ID: B18
New ID: B53
New ID: A61
New ID: A27
New ID: B49
New ID: A33
New ID: A75
New 

## next, we have data of all purchases

In [17]:
purchase_filename = 'transaction_data.csv'
df_transactions = pd.read_csv(purchase_filename)
#df_transactions.columns = names = ['id', 'user_id', 'marketing_rule', 'transaction_type', 'purchase_date', 'processed_at', 'gross_transaction_amount', 'discount_amount', 'earned_bonuses', 'redeemed_boneses', 'net_transaction_amount', 'connected_transaction', 'balance_1', 'name of the cashier', 'name of the user',  'login of the user',  'shop_name','shop_id','zero']

In [18]:
df_transactions

,id,user_id,transaction_type,purchase_date,processed_at,gross_transaction_amount,discount_amount,net_transaction_amount,shop_id
0,2676,9854,0,2023-03-13 13:32:46,2023-03-13,3097,41,3056,A85
1,20396,9910,0,2023-03-21 09:53:15,2023-03-21,917,82,835,A49
2,47074,1952,0,2023-03-24 12:48:15,2023-03-24,1820,54,1766,B68
3,81023,7402,0,2023-03-29 06:11:11,2023-03-29,9867,56,9811,A03
4,5448,6693,0,2023-03-25 07:21:58,2023-03-25,7164,49,7115,A03
...,...,...,...,...,...,...,...,...,...
995,16630,1748,0,2023-03-01 00:30:38,2023-03-01,8981,9,8972,A63
996,47463,6197,0,2023-03-23 11:00:09,2023-03-23,1489,9,1480,A84
997,3281,7468,0,2023-03-17 13:10:26,2023-03-17,3856,71,3785,B12
998,86309,9704,0,2023-03-17 15:44:32,2023-03-17,3290,27,3263,A62


In [19]:
df_purchase = df_transactions.loc[:, ['id', 'user_id', 'transaction_type', 'purchase_date',
       'processed_at', 'gross_transaction_amount', 'discount_amount','net_transaction_amount',  'shop_id' ]]

In [20]:
df_purchase

,id,user_id,transaction_type,purchase_date,processed_at,gross_transaction_amount,discount_amount,net_transaction_amount,shop_id
0,2676,9854,0,2023-03-13 13:32:46,2023-03-13,3097,41,3056,A85
1,20396,9910,0,2023-03-21 09:53:15,2023-03-21,917,82,835,A49
2,47074,1952,0,2023-03-24 12:48:15,2023-03-24,1820,54,1766,B68
3,81023,7402,0,2023-03-29 06:11:11,2023-03-29,9867,56,9811,A03
4,5448,6693,0,2023-03-25 07:21:58,2023-03-25,7164,49,7115,A03
...,...,...,...,...,...,...,...,...,...
995,16630,1748,0,2023-03-01 00:30:38,2023-03-01,8981,9,8972,A63
996,47463,6197,0,2023-03-23 11:00:09,2023-03-23,1489,9,1480,A84
997,3281,7468,0,2023-03-17 13:10:26,2023-03-17,3856,71,3785,B12
998,86309,9704,0,2023-03-17 15:44:32,2023-03-17,3290,27,3263,A62


In [21]:
df_purchase.dtypes

id                           int64
user_id                      int64
transaction_type             int64
purchase_date               object
processed_at                object
gross_transaction_amount     int64
discount_amount              int64
net_transaction_amount       int64
shop_id                     object
dtype: object

In [22]:

df_purchase.replace('\\N', None, inplace=True)

In [23]:
df_purchase

,id,user_id,transaction_type,purchase_date,processed_at,gross_transaction_amount,discount_amount,net_transaction_amount,shop_id
0,2676,9854,0,2023-03-13 13:32:46,2023-03-13,3097,41,3056,A85
1,20396,9910,0,2023-03-21 09:53:15,2023-03-21,917,82,835,A49
2,47074,1952,0,2023-03-24 12:48:15,2023-03-24,1820,54,1766,B68
3,81023,7402,0,2023-03-29 06:11:11,2023-03-29,9867,56,9811,A03
4,5448,6693,0,2023-03-25 07:21:58,2023-03-25,7164,49,7115,A03
...,...,...,...,...,...,...,...,...,...
995,16630,1748,0,2023-03-01 00:30:38,2023-03-01,8981,9,8972,A63
996,47463,6197,0,2023-03-23 11:00:09,2023-03-23,1489,9,1480,A84
997,3281,7468,0,2023-03-17 13:10:26,2023-03-17,3856,71,3785,B12
998,86309,9704,0,2023-03-17 15:44:32,2023-03-17,3290,27,3263,A62


In [24]:
quary_insert_into_purchase = '''
    INSERT INTO public.purchase (id, user_id, transaction_type, purchase_date, processed_at, gross_transaction_amount, discount_amount, net_transaction_amount, shop_id)
    VALUES (%(id)s, %(user_id)s, %(transaction_type)s, %(purchase_date)s, %(processed_at)s, %(gross_transaction_amount)s, %(discount_amount)s, %(net_transaction_amount)s, %(shop_id)s)
    ON CONFLICT (id)
    DO NOTHING
    RETURNING id;
'''

In [25]:
with connection:
    with connection.cursor() as cursor:
        for transaction_id, user_id, transaction_type, purchase_date, processed_at, gross_transaction_amount,discount_amount, net_transaction_amount, shop_id in df_purchase.values:
            data_keys = {
                'id': transaction_id,
                'user_id': user_id,
                'transaction_type' : transaction_type,
                'purchase_date': purchase_date,
                'processed_at': processed_at, 
                'gross_transaction_amount': gross_transaction_amount, 
                'discount_amount': discount_amount,
                'net_transaction_amount': net_transaction_amount,
                'shop_id': shop_id
            }
            try:
                cursor.execute(quary_insert_into_purchase, data_keys)
                #print(f"New ID: {cursor.fetchone()[0]}")
            except Exception as e:
                print(f"Error: {e}")
                break

## The last table is the data about the purchases items in every receipt. 

In [33]:
purchase_items_filename = 'items_data.csv'
df_purchase_items = pd.read_csv(purchase_items_filename)

In [34]:
df_purchase_items

,id,purchase_id,product_id,amount,price,discount,marketing_rule
0,2676-0,2676,2400,1,996,25,1
1,2676-1,2676,3067,1,375,16,0
2,2676-2,2676,2266,1,592,57,1
3,2676-3,2676,1566,1,795,6,1
4,2676-4,2676,1166,1,383,35,0
...,...,...,...,...,...,...,...
5351,3365-5,3365,2925,1,318,47,1
5352,3365-6,3365,1081,1,575,16,0
5353,3365-7,3365,1314,2,782,5,0
5354,3365-8,3365,2060,1,477,6,1


In [35]:
df_purchase_items = df_purchase_items.drop_duplicates()

In [36]:
quary_instert_into_purchase_items = '''
    INSERT INTO public.purchase_items (id, purchase_id, product_id, amount, price, dicsount, marketing_rule)
    VALUES (%(id)s, %(purchase_id)s, %(product_id)s, %(amount)s, %(price)s, %(dicsount)s, %(marketing_rule)s)
    ON CONFLICT (id)
    DO NOTHING
    RETURNING id;
'''

In [37]:
for purchase_items_id, purchase_id, product_id, amount, price, dicsount, marketing_rule in df_purchase_items.values:
            data_keys = {
                'id': purchase_items_id,
                'purchase_id': purchase_id,
                'product_id': product_id, 
                'amount': amount,
                'price': price,
                'dicsount': dicsount,
                'marketing_rule': marketing_rule
            }

In [38]:
df_purchase_items

,id,purchase_id,product_id,amount,price,discount,marketing_rule
0,2676-0,2676,2400,1,996,25,1
1,2676-1,2676,3067,1,375,16,0
2,2676-2,2676,2266,1,592,57,1
3,2676-3,2676,1566,1,795,6,1
4,2676-4,2676,1166,1,383,35,0
...,...,...,...,...,...,...,...
5351,3365-5,3365,2925,1,318,47,1
5352,3365-6,3365,1081,1,575,16,0
5353,3365-7,3365,1314,2,782,5,0
5354,3365-8,3365,2060,1,477,6,1


In [39]:
with connection:
    with connection.cursor() as cursor:
        for purchase_items_id, purchase_id, product_id, amount, price, dicsount, marketing_rule in df_purchase_items.values:
            data_keys = {
                'id': purchase_items_id,
                'purchase_id': purchase_id,
                'product_id': product_id, 
                'amount': amount,
                'price': price,
                'dicsount': dicsount,
                'marketing_rule': marketing_rule
            }
            if purchase_id is not None:
                try:
                    cursor.execute(quary_instert_into_purchase_items, data_keys)
                    #print(f"New ID: {cursor.fetchone()[0]}")
                except Exception as e:
                    print(f"Error: {e}")
                    break